In [1]:
import os

In [2]:
path = "../input/kla1task2/output"

In [3]:
labels =os.listdir(path)

In [4]:
d = {}
for x in labels :
    d[x] = len(os.listdir(path+'/'+ x))

In [5]:
len(d)

In [6]:
d

In [8]:
import numpy as np
import pandas as pd 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os
import matplotlib.pyplot as plt


In [9]:

filenames = os.listdir("../input/kla1task2/output")


In [10]:
files = []
categories = []
for filename in filenames:
    
    file  = "../input/kla1task2/output/" + filename
    
    for x in os.listdir(file):
        p = file +"//"+ x
        files.append(p)
        categories.append(filename)
#         print(filename,p)
    
df = pd.DataFrame({
    'filename': files,
    'category': categories
})

In [11]:
df.head()

In [12]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256

In [14]:
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1

In [15]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, Callback

def cnnblock(layer,filters,filtershape,dr = 0.25,deep = False , pool = True) :
    
    y = Conv2D(filters[0], filtershape, padding='same')(layer)
    y = Conv2D(filters[1], filtershape, padding='same')(y)
    y = Activation('relu')(y)
    if pool :
        y = MaxPooling2D(pool_size=(2, 2))(y)
    if deep :
        y = BatchNormalization()(y)
        y = Dropout(dr)(y)
    
    
    return y

In [29]:
inputs = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

x = cnnblock(inputs,[64,64],3,pool = True)

x = cnnblock(x,[32,32],3)

x = cnnblock(x,[16,16],3,deep = True)

x = Flatten()(x)

x = Dense(1000, activation='relu')(x)

x = Dense(300, activation='relu')(x)

x = Dense(28, activation='softmax')(x)

model = Model(inputs, x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.summary()

In [20]:
from sklearn.model_selection import train_test_split
train, validate = train_test_split(df, test_size=0.20, random_state=42)
train= train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
train.head()
plt.figure(figsize = (20,20))
train['category'].value_counts().plot.bar()

In [21]:
total_train = train.shape[0]
total_validate = validate.shape[0]
batch_size = 64

In [33]:
t_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

t_generator = t_datagen.flow_from_dataframe(
    train, 
    color_mode="grayscale",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)


In [35]:
v_datagen = ImageDataGenerator(rescale=1/255)

v_generator = v_datagen.flow_from_dataframe(
    validate, 
    color_mode="grayscale",
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [24]:
t_generator.class_indices

In [28]:
len(d)

In [25]:
d = t_generator.class_indices


In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
red = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

In [ ]:
epochs=25
modelhistory = model.fit(
    t_generator, 
    epochs=epochs,
    validation_data=v_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,callbacks=[red, checkpoint])
    


In [ ]:
plt.plot(modelhistory.history['accuracy'])
plt.plot(modelhistory.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# loss plot
plt.plot(modelhistory.history['loss'])
plt.plot(modelhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save_weights("task2(w).h5")
model.save("task2(m).h5")